In [6]:
!git clone https://github.com/Bisonai/mobilenetv3-tensorflow.git
%cd mobilenetv3-tensorflow 

Cloning into 'mobilenetv3-tensorflow'...
remote: Enumerating objects: 67, done.
remote: Total 67 (delta 0), reused 0 (delta 0), pack-reused 67
Unpacking objects: 100% (67/67), done.
/content/mobilenetv3-tensorflow


In [1]:
!pip3 list | grep tensorflow

tensorflow                    2.4.1          
tensorflow-addons             0.8.3          
tensorflow-datasets           4.0.1          
tensorflow-estimator          2.4.0          
tensorflow-gcs-config         2.4.0          
tensorflow-hub                0.11.0         
tensorflow-metadata           0.27.0         
tensorflow-privacy            0.2.2          
tensorflow-probability        0.12.1         


In [2]:
!pip3 uninstall tensorflow tensorflow-addons tensorflow-datasets tensorflow-estimator tensorflow-gcs-config tensorflow-hub tensorflow-metadata tensorflow-privacy tensorflow-probability -y

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
Uninstalling tensorflow-addons-0.8.3:
  Successfully uninstalled tensorflow-addons-0.8.3
Uninstalling tensorflow-datasets-4.0.1:
  Successfully uninstalled tensorflow-datasets-4.0.1
Uninstalling tensorflow-estimator-2.4.0:
  Successfully uninstalled tensorflow-estimator-2.4.0
Uninstalling tensorflow-gcs-config-2.4.0:
  Successfully uninstalled tensorflow-gcs-config-2.4.0
Uninstalling tensorflow-hub-0.11.0:
  Successfully uninstalled tensorflow-hub-0.11.0
Uninstalling tensorflow-metadata-0.27.0:
  Successfully uninstalled tensorflow-metadata-0.27.0
Uninstalling tensorflow-privacy-0.2.2:
  Successfully uninstalled tensorflow-privacy-0.2.2
Uninstalling tensorflow-probability-0.12.1:
  Successfully uninstalled tensorflow-probability-0.12.1


In [3]:
!pip3 install -q tensorflow_gpu==1.13.1

     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 368kB 47.1MB/s 
     |████████████████████████████████| 3.2MB 49.8MB/s 
     |████████████████████████████████| 51kB 9.4MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1


In [8]:
!pip3 install tensorflow_datasets

     |████████████████████████████████| 3.7MB 16.5MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 


In [10]:
!pip3 list | grep tensorflow

tensorflow-datasets           4.2.0          
tensorflow-estimator          1.13.0         
tensorflow-gpu                1.13.1         
tensorflow-metadata           0.27.0         


In [9]:
import tensorflow as tf

from mobilenetv3_factory import build_mobilenetv3
from datasets import build_dataset



***************************************************************
Failed to import TensorFlow. Please note that TensorFlow is not installed by default when you install TFDS. This allow you to choose to install either `tf-nightly` or `tensorflow`. Please install the most recent version of TensorFlow, by following instructions at https://tensorflow.org/install.
***************************************************************




ModuleNotFoundError: ignored

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

In [ ]:
# Parameters
name = "mnist"
train_batch_size = 128
valid_batch_size = 128
height = 64
width = 64
model_type = "small"
l2_reg = 0.0001
lr = 0.01
width_multiplier = 1.0
optimizer = tf.keras.optimizers.RMSprop(lr)
num_epoch = 5

In [ ]:
dataset = build_dataset(
        name=name,
        shape=[height,width],
        train_batch_size=train_batch_size,
        valid_batch_size=valid_batch_size
        )

W0709 08:58:01.947725 139961290983296 deprecation.py:323] From /content/mobilenetv3-tensorflow/datasets.py:34: DatasetV1.output_shapes (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
W0709 08:58:01.957156 139961290983296 deprecation_wrapper.py:119] From /content/mobilenetv3-tensorflow/datasets.py:52: The name tf.image.resize_images is deprecated. Please use tf.image.resize instead.



In [ ]:
model = build_mobilenetv3(
        model_type,
        input_shape=(height, width, dataset["channels"]),
        num_classes=dataset["num_classes"],
        width_multiplier=width_multiplier,
        l2_reg=l2_reg,
    )

W0709 08:58:02.015088 139961290983296 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.compile(
        optimizer= optimizer,
        loss="categorical_crossentropy",
        metrics=["accuracy"],)

In [ ]:
model.fit(
        dataset["train"].make_one_shot_iterator(),
        steps_per_epoch=(dataset["num_train"]//train_batch_size)+1,
        epochs=num_epoch,
        validation_data=dataset["test"],
        validation_steps=(dataset["num_test"]//valid_batch_size)+1,
    )

W0709 08:58:07.003938 139961290983296 deprecation.py:323] From <ipython-input-8-bf59adf0d72b>:2: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
W0709 08:58:07.471397 139961290983296 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
469/469 [==============================] - 40s 86ms/step - loss: 1.4966 - acc: 0.5967 - val_loss: 1.1349 - val_acc: 0.7972
Epoch 2/5
469/469 [==============================] - 32s 68ms/step - loss: 0.2875 - acc: 0.9496 - val_loss: 0.2680 - val_acc: 0.9643
Epoch 3/5
469/469 [==============================] - 32s 67ms/step - loss: 0.1977 - acc: 0.9677 - val_loss: 1.6569 - val_acc: 0.5516
Epoch 4/5
469/469 [==============================] - 32s 68ms/step - loss: 0.1646 - acc: 0.9745 - val_loss: 0.2465 - val_acc: 0.9617
Epoch 5/5
469/469 [==============================] - 32s 68ms/step - loss: 0.1492 - acc: 0.9775 - val_loss: 0.1876 - val_acc: 0.9668
